#OAuth Exercise

In this exercise we will try to scrape twitter data and do a tf-idf analysis on that (src-uwes twitter analysis). We will need OAuth authentication, and we will follow a similar approach as detailed in the yelp analysis notebook. 

In [2]:
import oauth2 as oauth
import urllib2 as urllib
import pickle
import json
import math
import nltk
import string
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import *

We will now need twitter api access. The following steps as available online will help you set up your twitter account and access the live 1% stream.

1. Create a twitter account if you do not already have one.
2. Go to https://dev.twitter.com/apps and log in with your twitter credentials.
3. Click "Create New App"
4. Fill out the form and agree to the terms. Put in a dummy website if you don't have one you want to use.
5. On the next page, click the "API Keys" tab along the top, then scroll all the way down until you see the section "Your Access Token"
6. Click the button "Create My Access Token". You can Read more about Oauth authorization online. 

Save the details of api_key, api_secret, access_token_key, access_token_secret in your vaule directory and load it in the notebook as shown in yelpSample notebook.

In [3]:
# We need to define the following variables
#api_key = #<get api key> 
#api_secret = #<get api secret>
#access_token_key = #<get your access token key here>"
#access_token_secret = #<get your access token secret here>

#defining them right here is not safe. insteadm create a file in a different directory
# (I use ~/VaultDSE) and in it put a file called, say, twitterkeys.py whose
# content is:
#api_key = #<get api key>
#api_secret = #<get api secret>
#access_token_key = #<get your access token key here>"
#access_token_secret = #<get your access token secret here>
#
#def getkeys():
#    return api_key,api_secret,access_token_key,access_token_secret

# then use the following commands

import sys
sys.path.append('/Users/RyanRiopelle/Documents/DSE/rriopell/DSE200/day_5_mining_the_Social_web/VaultDSE')
import twitterKeys
api_key,api_secret,access_token_key,access_token_secret=twitterKeys.getkeys()

_debug = 0

oauth_token    = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"

http_handler  = urllib.HTTPHandler(debuglevel=_debug)
https_handler = urllib.HTTPSHandler(debuglevel=_debug)

Below is a twitter request method which will use the above user logins to sign, and open a twitter stream request

In [4]:
def getTwitterStream(url, method, parameters):
  req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=http_method,
                                             http_url=url, 
                                             parameters=parameters)

  req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)

  headers = req.to_header()

  if http_method == "POST":
    encoded_post_data = req.to_postdata()
  else:
    encoded_post_data = None
    url = req.to_url()

  opener = urllib.OpenerDirector()
  opener.add_handler(http_handler)
  opener.add_handler(https_handler)

  response = opener.open(url, encoded_post_data)

  return response

We can use the above function to request a response as follows

In [9]:
url = "https://stream.twitter.com/1/statuses/sample.json"
parameters = []
response = getTwitterStream(url, "GET", parameters)
print response

<addinfourl at 4360247416 whose fp = <socket._fileobject object at 0x108199ed0>>


In [8]:
#Now we will test the above function for a sample data provided by twitter stream here -  
url = "https://stream.twitter.com/1/statuses/sample.json"

Write a function which will take a url and return the top 10 lines returned by the twitter stream

** Note ** The response returned needs to be intelligently parsed to get the text data which correspond to actual tweets. This part can be done in a number of ways and you are encouraged to try different approaches to parse the response data.

In [16]:
url="https://api.twitter.com/1.1/search/tweets.json?q="
searchString="Syria"
count="200"

def WriteFileTwitterSearch(url, searchString,count):
    urlSearch=url+searchString+'&count='+count
    parameters = []
    response = getTwitterStream(urlSearch, "GET", parameters)
    a= response.readline() 
    dic=json.loads(a)
    tweets=dic['statuses']
    tweetdic={}
    twCount=0
    for t in tweets:
        tweetdic['Tweet%i'%twCount]=t['text']
        twCount=twCount+1
    #json.dump(tweetdic, open("twitterStreamFile.txt",'wb')) 
    x=1
    while x<11:
        print "Tweet%i:"%x, tweetdic['Tweet%i'%x]
        x=x+1
    
WriteFileTwitterSearch(url, searchString,count)

 Tweet1: Malaysians, Singaporeans nabbed before leaving for Syria had no prior terror links - Inquirer.n... https://t.co/ETD9sAnPlf #ISIS
Tweet2: RT @edwardedark: Most schools are closed in west Aleppo today, due to last 2 days of terrorist rebel shelling that murdered &amp; wounded dozen…
Tweet3: RT @PhilGreaves01: Hey Prof, @joshua_landis you're still uncritically promoting fairytales of "Syria Direct", entirely funded by USGov. htt…
Tweet4: RT TheOnlyMan2Know Why are we here? https://t.co/UpipbDs5hU #Hungary #Syria #Palestine #Israel #Iran #European #Europe
Tweet5: ثورة الشام بدأت باسم الله وصمدت بفضل الله ولن تنتصر إلا أن يشاء الله، لا منة عليها لأحد إلا لبارئها
أحسنوا ظنكم بالله فحاشاه أن يخيب سعيكم
Tweet6: RT @memorte2023: Syrian opposition has agreed in Riyadh meeting to form a committee of 23 members https://t.co/SvPCM9Y0rw #SaudiArabia #Syr…
Tweet7: RT @HK2307: Acc to @dianadarke #US needs to grasp important difference bw #IS &amp; AlQaeda in #Syria #Nusra;
JN respects minorit

In [ ]:
#We can also request twitter stream data for specific search parameters as follows
url= "https://api.twitter.com/1.1/search/tweets.json?q="+search_query
#get twitter stream first
#this one looks like adictionary in a dictionary, still want to do read loads
#do in to statuses, which is a list, inside the list is text


Call the fetchData function to fetch latest live stream data for following search queries and output the first 5 lines

1. "UCSD"
2. "Donald Trump"
3. "Syria"

In [18]:
#url = "https://api.twitter.com/1.1/search/tweets.json?q="+"UCSD"
def fetchDataTwitterSearch(url, searchString,count):
    urlSearch=url+searchString+'&count='+count
    parameters = []
    response = getTwitterStream(urlSearch, "GET", parameters)
    a= response.readline()
    dic=json.loads(a)
    dic1=dic['statuses']
    for i in dic1:
        print i['text']
    #return dic1
fetchDataTwitterSearch("https://api.twitter.com/1.1/search/tweets.json?q=", "UCSD","5")

Newsworthy: read this post by @GPS_UCSD professor @StephanHaggard on #NorthKorea, #Iran, #nuclearweapons https://t.co/4F7P9ex04M
https://t.co/docK6cmgzR Depressed from Ochem #UCSD #reddit
RT @UC_net: #ClimateChange facts you need to know, courtesy of #UCSD @Scripps_Ocean expert — https://t.co/w2pzhc1nUr #COP21 https://t.co/yq…
Coursera Data Science Specializations - John Hopkins Data Sci vs. UCSD Big Data via /r/datascience https://t.co/5gYniwydXw
RT @UCSDHealth: Our Comprehensive Stroke Center earns gold from @American_Heart &amp; @American_Stroke! https://t.co/4kRKbIEiJK https://t.co/JO…


In [19]:
fetchDataTwitterSearch("https://api.twitter.com/1.1/search/tweets.json?q=", "Donald Trump", "5")

RT @Eating: RT if you think this mozzarella stick would be a better president than Donald Trump. https://t.co/xwXui5MeYr
RT @bbcnickrobinson: Petition calling for Donald Trump to be barred from UK now most popular ever on govt website
Just putting this out there. I will never vote for Donald Trump. Even against Hillary. Trump is not a republican. #republicansagainsttrump
RT @IrishTimes: Dubai firm strips Donald Trump’s image and name from golf complex https://t.co/DQnyhOqbW3 via @IrishTimesSport https://t.co…
RT @Independent: Watch Sacha Baron Cohen bring back 'Borat’ just to mock Donald Trump https://t.co/9dowafz654 https://t.co/M8Jk9GhWoU


In [20]:
fetchDataTwitterSearch("https://api.twitter.com/1.1/search/tweets.json?q=", "Syria","5")

RT @CSIS: US Special Forces alone are not enough to turn the tide against ISIS: https://t.co/yZQjJyrDMN https://t.co/nKD2zwlMlx
RT @WeAsk4Justice: @RevolutionSyria: أوقفوا مجرم الحرب بوتين، من ارتكاب الجرائم في سوريا.
Stop this Russian mass killer
#Syria https://t.co…
RT @horrificstory: Syria 2010 vs Syria 2015. What is there left to bomb? https://t.co/NAb35yh0CO


### TF-IDF###

tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.It is among the most regularly used statistical tool for word cloud analysis. You can read more about it online (https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

We base our analysis on the following

1. The weight of a term that occurs in a document is simply proportional to the term frequency
2. The specificity of a term can be quantified as an inverse function of the number of documents in which it occurs

For this question we will perform tf-idf analysis o the stream data we retrieve for a given search parameter. Perform the steps below

1. use the twitterreq function to search for the query "syria" and save the top 200 lines in the file twitterStream.txt
2. load the saved file and output the count of occurrences for each term. This will be your term frequency
3. Calculate the inverse document frequency for each of the term in the output above.
4. Divide the term frequency for each of the term by corresponding inverse document frequency.
5. Sort the terms in the descending order based on their term freq/inverse document freq scores 
6. Print the top 10 terms.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

#text_file = open("twitterStreamFile.txt", "r")
dict_file = json.load (open("twitterStreamFile.txt") )
token_dict={}
stemmer = PorterStemmer()

def get_tokens(input_text):
    text = input_text.encode('ascii','ignore')
    lowers = text.lower()
    no_punctuation = lowers.translate(None, string.punctuation)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens
    #print token

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

for key in dict_file:
    text = dict_file[key].encode('ascii','ignore')
    lowers = text.lower()
    no_punctuation = lowers.translate(None, string.punctuation)
    token_dict[key] = no_punctuation

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

text_file = json.load (open("twitterStreamFile.txt") )
combined= unicode.join(u'',text_file.values())

str = combined 
response = tfidf.transform([str])
#print response

answerlist=[]

feature_names = tfidf.get_feature_names()
for col in response.nonzero()[1]:
    word_value = (feature_names[col], response[0, col])
    answerlist.append(word_value)
    #print feature_names[col], ' - ', response[0, col]

top10=sorted(answerlist, key=lambda student: student[1])
print top10[-10:]

[(u'state', 0.076336076305691211), (u'iraq', 0.076336076305691211), (u'world', 0.081181991353744959), (u'refuge', 0.081181991353744959), (u'assad', 0.081181991353744959), (u'thi', 0.095429728354270965), (u'isi', 0.10937850996316029), (u'amp', 0.14303045961686084), (u'syria', 0.2858680416727703), (u'http', 0.79206674534044896)]
